<a href="https://colab.research.google.com/github/jlab-sensing/MFC_Modeling/blob/main/SNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  SNN Models
###  In order to run the code in this notebook, you must download the `ucscMFCDataset` directory and `stanfordMFCDataset.zip`, which expands into the directory `rocket4`, from [Hugging Face](https://huggingface.co/datasets/adunlop621/Soil_MFC/tree/main), and store them in the same directory as this notebook. You can also find several pretrained models in the at this link, with the naming conventions described in the [README](https://github.com/jlab-sensing/MFC_Modeling#:~:text=Repository%20files%20navigation-,README,-MFC_Modeling)

In [3]:
%pip install --upgrade hepml
%pip install arrow
%pip install keras_lr_finder
%pip install pandas
%pip install snntorch --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# reload modules before executing user code
#%load_ext autoreload
# reload all modules every time before executing Python code
#%autoreload 2
# render plots in notebook

# Misc imports
%matplotlib inline
import datetime
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from hepml.core import plot_regression_tree
sns.set(color_codes=True)
sns.set_palette(sns.color_palette("muted"))
import random
import statistics

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

# torch imports
import torch
import torch.nn as nn

# snnTorch/torch imports
import snntorch as snn
from snntorch import functional as SF
import snntorch.spikeplot as splt
from torch.utils.data import DataLoader, TensorDataset


##  Load and Format Dataset 1

### Remember to download `stanfordMFCDataset.zip`, which expands into the directory `rocket4`, from [Hugging Face](https://huggingface.co/datasets/adunlop621/Soil_MFC/tree/main), and store it in the same directory as this notebook before executing the following code.

In [29]:
#Load teros data
import glob
teros_files = glob.glob("rocket4/TEROSoutput*.csv")
X = pd.DataFrame()
for f in teros_files:
  try:
    csv = pd.read_csv(f, index_col=False).dropna()
    X = pd.concat([X, csv])
  except:
    continue

In [6]:
#Load power data
power_files = glob.glob("rocket4/soil*.csv")
y = pd.DataFrame()
for f in sorted(power_files, key=lambda x: int(x.split('.')[0].split('_')[-1])):
#in power_files:
  try:
    csv = pd.read_csv(f, on_bad_lines='skip', skiprows=10).dropna(how='all')
    csv = csv.rename({'Unnamed: 0': 'timestamp'}, axis='columns')
    y = pd.concat([y,csv])
  except:
    continue
y["timestamp"] = y["timestamp"].round(decimals = 1)

In [7]:
#Convert current to amps, voltage to volts
y["I1L [10pA]"] = np.abs(y["I1L [10pA]"] * 1E-11)
y["V1 [10nV]"] = np.abs(y["V1 [10nV]"] * 1E-8)
y["I1H [nA]"] = np.abs(y["I1H [nA]"] * 1E-9)

In [8]:
#Sort data by timestamp, convert to datetime
X = X.sort_values(['timestamp'])
y = y.sort_values(['timestamp'])
X['timestamp'] = pd.to_datetime(X['timestamp'], unit='s')
y['timestamp'] = pd.to_datetime(y['timestamp'], unit='s')

#Merge data by timestamp
uncut_df = pd.merge_asof(left=X,right=y,direction='nearest',tolerance=pd.Timedelta('1 sec'), on = 'timestamp').dropna(how='all')

#Isolate data from cell0
df = uncut_df.loc[uncut_df['sensorID'] == 0]

#Localize timestamp
df.timestamp = df.timestamp.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')

/var/folders/1g/jt7x_x9n6j1f3317_dwv3k9m0000gn/T/ipykernel_40442/3455547848.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.timestamp = df.timestamp.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')


In [9]:
#Use only data from after deployment date
#df = df.loc[(df['timestamp'] > '2021-09-24') & (df['timestamp'] < '2021-10-15')] #Future of Clean Computing Graph
#df = df.loc[(df['timestamp'] > '2021-06-24') & (df['timestamp'] < '2021-07-02')]
#df = df.loc[(df['timestamp'] > '2021-06-18')] #Two weeks after deployment
df = df.loc[(df['timestamp'] > '2021-06-04')] #Deployment date
#df = df.loc[(df['timestamp'] > '2021-06-25') & (df['timestamp'] < '2021-06-26')] #Small training set

#Power drop
#df = df.loc[(df['timestamp'] > '2021-11-01') & (df['timestamp'] < '2021-11-22')]

#Drop data outages
df = df.drop(df[(df.timestamp > '2021-11-11') & (df.timestamp < '2021-11-22 01:00:00')].index)
df = df.drop(df[(df.timestamp > '2022-01-27')].index)
#df = df.set_index('timestamp')
df = df[:-1]

In [10]:
df = df.set_index('timestamp')

In [11]:
#Get time since deployement
df['tsd'] = (df.index - df.index[0]).days
df['hour'] = (df.index).hour

In [12]:
#Calculate power
df["power"] = np.abs(np.multiply(df.iloc[:, 7], df.iloc[:, 8]))
#df["power"] = np.abs(np.multiply(df["I1L [10pA]"], df["V1 [10nV]"]))

#Convert to nW
df['power'] = df['power']*1E9

In [13]:
#Convert to 10 nanoamps, 10 microvolts
df["I1L [10pA]"] = np.abs(df["I1L [10pA]"] * 1E8)
df["V1 [10nV]"] = np.abs(df["V1 [10nV]"] * 1E5)
df["I1H [nA]"] = np.abs(df["I1H [nA]"] * 1E8)

In [14]:
df = df.reset_index()

In [15]:
#Add power time series
df['power - 1h'] = df['power'].shift(1).dropna()
df['power - 2h'] = df['power'].shift(2).dropna()
df['power - 3h'] = df['power'].shift(3).dropna()
#df['power - 2h'] = df['power'].shift(2).dropna()
#df['previous_power - 3'] = df['power'].shift(3).dropna()
#df['previous_power - 4'] = df['power'].shift(4).dropna()

#Add teros time series
df['EC - 1h'] = df['EC'].shift(1).dropna()
df['EC - 2h'] = df['EC'].shift(2).dropna()
df['EC - 3h'] = df['EC'].shift(3).dropna()

df['temp - 1h'] = df['temp'].shift(1).dropna()
df['temp - 2h'] = df['temp'].shift(2).dropna()
df['temp - 3h'] = df['temp'].shift(3).dropna()

df['raw_VWC - 1h'] = df['raw_VWC'].shift(1).dropna()
df['raw_VWC - 2h'] = df['raw_VWC'].shift(2).dropna()
df['raw_VWC - 3h'] = df['raw_VWC'].shift(3).dropna()

#Add voltage and current time series
df['V1 - 1h'] = df['V1 [10nV]'].shift(1).dropna()
df['V1 - 2h'] = df['V1 [10nV]'].shift(2).dropna()
df['V1 - 3h'] = df['V1 [10nV]'].shift(3).dropna()

df['I1L - 1h'] = df['I1L [10pA]'].shift(1).dropna()
df['I1L - 2h'] = df['I1L [10pA]'].shift(2).dropna()
df['I1L - 3h'] = df['I1L [10pA]'].shift(3).dropna()

df['I1H - 1h'] = df['I1H [nA]'].shift(1).dropna()
df['I1H - 2h'] = df['I1H [nA]'].shift(2).dropna()
df['I1H - 3h'] = df['I1H [nA]'].shift(3).dropna()
df = df.dropna()

In [16]:
#df = df.rename(columns={'power': 'power [μW]'})
df = df.rename(columns={'I1L [10pA]': 'Current (uA)', 'V1 [10nV]' : 'Voltage (mV)', 'power' : 'Power (uW)'})
df = df.set_index('timestamp')

## Specify Device so we can use GPU

In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Model Architecture

In [19]:
beta = 0.9

# old design network
# model = Sequential()
# model.add(LSTM(200, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(3))
# model.compile(loss=quantile_loss, metrics=['mape'], optimizer='adam')

# Define Network
class Net(nn.Module):
    def __init__(self, num_inputs, num_steps):
        super().__init__()

        self.num_inputs = num_inputs
        self.num_steps = num_steps

        num_hidden1 = 200

        # layer 1
        self.slstm1 = snn.SLSTM(num_inputs, num_hidden1, threshold = 0.25)

        # layer 2
        self.fc1 = torch.nn.Linear(in_features=num_hidden1, out_features=100)
        self.lif1 = snn.Leaky(beta=beta, threshold = 0.5)

        # randomly initialize decay rate for output neuron
        beta_out = random.uniform(0.5, 1)

        # layer 2
        self.fc2 = torch.nn.Linear(in_features=100, out_features=3)
        self.lif2 = snn.Leaky(beta=beta_out, learn_beta=True, reset_mechanism="none")


    def forward(self, x):
        # Initialize hidden states and outputs at t=0
        syn1, mem1 = self.slstm1.reset_mem()
        mem2 = self.lif1.reset_mem()
        mem3 = self.lif2.reset_mem()

        # Record the final layer
        spk1_rec = []
        spk2_rec = []
        spk3_rec = []
        mem_rec = []

        for step in range(self.num_steps):
            spk1, syn1, mem1 = self.slstm1(x.flatten(1), syn1, mem1)
            spk2, mem2 = self.lif1(self.fc1(spk1), mem2)
            spk3, mem3 = self.lif2(self.fc2(spk2), mem3)

            # Append the Spike and Membrane History
            spk1_rec.append(spk1)
            spk2_rec.append(spk2)
            spk3_rec.append(spk3)
            mem_rec.append(mem3)

        return torch.stack(spk1_rec), torch.stack(spk2_rec), torch.stack(spk3_rec), torch.stack(mem_rec)

# Loading Teacher Predictions from X_train

In [99]:
from keras.models import load_model

# Set parameters
batchsize_list = [300, 150, 50, 20, 8]
time_frame_list = ['3min', '5min', '15min', '30min', '60min']
time_frame_seconds_list = [180, 300, 900, 1800, 3600]
n = 0

snn_power_mape_list = []
snn_volt_mape_list = []
snn_curr_mape_list = []

# Dictionary to store mv variables
mv_dict = {}

for j in range(len(batchsize_list)):
    batchsize = batchsize_list[j]
    time_frame = time_frame_list[j]
    time_frame_seconds = time_frame_seconds_list[j]

    X = pd.concat([df["power - 1h"], df["power - 2h"], df["power - 3h"], 
                   df["V1 - 1h"], df["V1 - 2h"], df["V1 - 3h"], 
                   df["I1L - 1h"], df["I1L - 2h"], df["I1L - 3h"], 
                   df["EC - 1h"], df["EC - 2h"], df["EC - 3h"], 
                   df["raw_VWC - 1h"], df["raw_VWC - 2h"], df["raw_VWC - 3h"], 
                   df["temp - 1h"], df["temp - 2h"], df["temp - 3h"], 
                   df["tsd"], df["hour"]], axis=1)
    y = pd.concat([df["Power (uW)"], df['Voltage (mV)'], df['Current (uA)']], axis=1)

    # Normalize Data
    X_normalized = ((X - X.min()) / (X.max() - X.min()))

    # Split train and test sets
    X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
    y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

    X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
    y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)

    # Resample data

    X_train = X_train.resample(time_frame).mean().dropna()
    y_train = y_train.resample(time_frame).mean().dropna()

    X_valid = X_valid.resample(time_frame).mean().dropna()
    y_valid = y_valid.resample(time_frame).mean().dropna()

    X_test = X_test.resample(time_frame).mean().dropna()
    y_test = y_test.resample(time_frame).mean().dropna()

    # Define mv variable for the current time frame
    mv_dict[time_frame] = y_test

    # Reshape data
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_valid = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Convert to tensor
    X_train = torch.tensor(X_train)
    y_train = torch.tensor(y_train.values)
    X_valid = torch.tensor(X_valid)
    y_valid = torch.tensor(y_valid.values)
    X_test = torch.tensor(X_test)
    y_test = torch.tensor(y_test.values)

    # Make datasets
    train_dataset = TensorDataset(X_train, y_train)
    valid_dataset = TensorDataset(X_valid, y_valid)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=False)
    valid_loader = DataLoader(valid_dataset, batch_size=batchsize, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

    num_steps = 50
    num_inputs = X_train.shape[2]

    # Create new instance of the SNN Class
    model = Net(num_inputs, num_steps).to(device)

    file = 'trained_models/snn_' + time_frame + '_quant50.pth'
    print(file)

    checkpoint = torch.load(file, map_location=torch.device('cpu'), weights_only=True)
    model.load_state_dict(checkpoint['state_dict'])

    model.eval()
    actuals = []
    predictions = []


    with torch.no_grad():
        for data, targets in test_loader:
            # Prepare data
            data = data.to(device).float()
            targets = targets.to(device).float()

            _, _, _, output = model(data)

            output = output.cpu().squeeze(1).detach()
            actuals.append(targets)
            predictions.append(output[-1])

    # Convert lists to tensors
    actuals = torch.cat(actuals, dim=0)
    predictions = torch.cat(predictions, dim=0)

    mv = mv_dict[time_frame]
    print(time_frame)
    mv["power_pred_med_" + time_frame] = predictions[:, 0].numpy()
    mv["voltage_pred_med_" + time_frame] = predictions[:, 1].numpy()
    mv["current_pred_med_" + time_frame] = predictions[:, 2].numpy()

    print(f'Voltage overestimation rate for {time_frame}: %.3f%%' % (
        (mv['Voltage (mV)'].values <= mv["voltage_pred_med_" + time_frame]).mean() * 100))
    print(f"Test MAPE power ({time_frame}): %3f" % MAPE(mv['Power (uW)'].values.ravel(), mv["power_pred_med_" + time_frame]))
    print(f"Test MAPE voltage ({time_frame}): %3f" % MAPE(mv['Voltage (mV)'], mv["voltage_pred_med_" + time_frame]))
    print(f"Test MAPE current ({time_frame}): %3f" % MAPE(mv['Current (uA)'], mv["current_pred_med_" + time_frame]))


    V_actual = mv['Voltage (mV)'].mean()
    V_pred = mv["voltage_pred_med_" + time_frame].mean()
    print(f'Predicted vs. Actual Total Voltage Percent Difference ({time_frame}): %.3f%%' % (
        (V_pred - V_actual) * 100 / V_actual))

trained_models/snn_3min_quant50.pth
3min
Voltage overestimation rate for 3min: 85.231%
Test MAPE power (3min): 0.537724
Test MAPE voltage (3min): 0.290864
Test MAPE current (3min): 0.316784
Predicted vs. Actual Total Voltage Percent Difference (3min): 21.857%
trained_models/snn_5min_quant50.pth
5min
Voltage overestimation rate for 5min: 73.179%
Test MAPE power (5min): 0.394733
Test MAPE voltage (5min): 0.214422
Test MAPE current (5min): 0.241269
Predicted vs. Actual Total Voltage Percent Difference (5min): 12.812%
trained_models/snn_15min_quant50.pth
15min
Voltage overestimation rate for 15min: 80.689%
Test MAPE power (15min): 0.328628
Test MAPE voltage (15min): 0.257437
Test MAPE current (15min): 0.227218
Predicted vs. Actual Total Voltage Percent Difference (15min): 17.903%
trained_models/snn_30min_quant50.pth
30min
Voltage overestimation rate for 30min: 82.230%
Test MAPE power (30min): 0.317190
Test MAPE voltage (30min): 0.264197
Test MAPE current (30min): 0.169668
Predicted vs. Act

In [100]:
print(mv_dict['15min'].keys())

Index(['Power (uW)', 'Voltage (mV)', 'Current (uA)', 'power_pred_med_15min',
       'voltage_pred_med_15min', 'current_pred_med_15min'],
      dtype='object')


We have populated *mv_dict* with the predictions for power, voltage, and current at each tiem step 3min, 5min, 15min, 30min, and 60min. For this implementation, we focused on using the 5min predictions to predict for 15mins. 

In this next section, we use *mv_dict['5min']* as the teacher prediction to assist the forecasting for 15mins. 

# Student Model

## 1. Defining Loss Functions

In [101]:
def quantile_loss(y_true, y_pred, quantile=0.5):
    error = y_true - y_pred
    loss = torch.mean(torch.max(quantile * error, (quantile - 1) * error))
    return loss

def distillation_loss(y_teacher, y_pred):
  error = y_teacher - y_pred
  loss = torch.mean((error) ** 2)
  return loss

def combined_loss(y_true, y_pred, y_teacher, quantile=0.5, alpha=1):
  if alpha > 1 or alpha < 0:
    raise ValueError("alpha must be between 0 and 1")
  return ((alpha) * quantile_loss(y_true, y_pred, quantile)) + ((1-alpha) * distillation_loss(y_teacher, y_pred))

## 2. Preparing Student Training Data

### 2.1 Change the values to alter the Student-Teacher forecasting relationship 

In [106]:
# valid teacher timeframes: 3min, 5min, 10min, 15min, 30min, 60min
valid_timeframes = ['3min', '5min', '10min', '15min', '30min', '60min']
teacher_timeframe = valid_timeframes[1]
#valid student timeframes: 3min, 5min, 10min, 15min, 30min, 60min (student timeframe must be <= teacher timeframe)
student_timeframe = valid_timeframes[5]

In [107]:
teacher_5min_preds = mv_dict[teacher_timeframe]
teacher_5min_preds_df = pd.DataFrame(
    teacher_5min_preds,
    columns=[
        "power_pred_med_" + teacher_timeframe,
        "voltage_pred_med_" + teacher_timeframe,
        "current_pred_med_" + teacher_timeframe
    ]
)

X_normalized = ((X - X.min()) / (X.max() - X.min()))

# Using X, y from Teacher model
X_train, X_test = train_test_split(X_normalized, test_size=0.3, shuffle=False)
y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False)

# Split the testing set into validation and final test sets (50/50)
X_valid, X_test = train_test_split(X_test, test_size=0.5, shuffle=False)
y_valid, y_test = train_test_split(y_test, test_size=0.5, shuffle=False)



# Resample data to 15-minute intervals
X_train_final = X_train.resample(student_timeframe).mean().dropna()
y_train_final = y_train.resample(student_timeframe).mean().dropna()
X_valid_final = X_valid.resample(student_timeframe).mean().dropna()
y_valid_final = y_valid.resample(student_timeframe).mean().dropna()
X_test_final = X_test.resample(student_timeframe).mean().dropna()
y_test_final = y_test.resample(student_timeframe).mean().dropna()


# Align predictions to data
# Filter 5-minute predictions to keep those 5 minutes before 15-minute intervals
valid_timestamps = y_train_final.index

teacher_5min_preds_df = teacher_5min_preds_df.loc[teacher_5min_preds_df.index.isin(valid_timestamps - pd.Timedelta(minutes=5))]
# Reassign the prediction timestamps to align with the current data timestamp
teacher_5min_preds_df.index = teacher_5min_preds_df.index + pd.Timedelta(minutes=5)


# X_train_student = pd.concat([X_train, teacher_5min_preds_df], axis=1)
y_train_final = pd.concat([y_train_final, teacher_5min_preds_df], axis=1)

# Reshape data for LSTM input
X_train_final = X_train_final.values.reshape((X_train_final.shape[0], 1, X_train_final.shape[1]))
X_valid_final = X_valid_final.values.reshape((X_valid_final.shape[0], 1, X_valid_final.shape[1]))
X_test_final = X_test_final.values.reshape((X_test_final.shape[0], 1, X_test_final.shape[1]))


X_train_final = torch.tensor(X_train_final).float()
y_train_final = torch.tensor(y_train_final.values).float()
X_valid_final = torch.tensor(X_valid_final).float()
y_valid_final = torch.tensor(y_valid_final.values).float()
X_test_final = torch.tensor(X_test_final).float()
y_test_final = torch.tensor(y_test_final.values).float()


# Clean X_train_final tensor
nan_mask = torch.isnan(y_train_final)  # Identify NaN values
non_nan_values = y_train_final[~nan_mask]  # Filter out NaN values for mean calculation
mean_value = torch.mean(non_nan_values)  # Compute mean of valid values

# Replace NaNs with the computed mean
tensor_cleaned = y_train_final.clone()
tensor_cleaned[nan_mask] = mean_value

# Reassign cleaned tensor back if desired
y_train_final = tensor_cleaned

print("Updated X Shape: Rows " + str(X_train_final.shape[0]) + " Columns " + str(X_train_final.shape[1]))
print("X_train shape: ", X_train_final.shape)
print("y_train shape: ", y_train_final.shape)



Updated X Shape: Rows 2576 Columns 1
X_train shape:  torch.Size([2576, 1, 20])
y_train shape:  torch.Size([2576, 6])


## 3. Run Training on Student Model

In [ ]:
print("Performing alpha sweep on student model")
for i in np.arange(0, 1.1, 0.1):
    alpha = i
    power_mape = []
    voltage_mape = []
    current_mape = []

    E_actual_list = []
    E_pred_list = []

    max_act_list = []
    pred_act_list = []
    succ_act_list = []

    pred_act_naive_list = []
    false_act_naive_list = []
    succ_act_naive_list = []

    # initialize histories
    loss_hist = []
    avg_loss_hist = []
    acc_hist = []
    mape_hist = []


    # Set parameters
    batch_size = 32  # Adjust as needed
    num_epochs = 10
    learning_rate = 1e-2
    beta = 0.8  # For quantile loss

    # Create datasets
    train_dataset_final = TensorDataset(X_train_final, y_train_final)
    valid_dataset_final = TensorDataset(X_valid_final, y_valid_final)
    test_dataset_final = TensorDataset(X_test_final, y_test_final)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset_final, batch_size=batch_size, shuffle=False)
    valid_loader = DataLoader(valid_dataset_final, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset_final, batch_size=batch_size, shuffle=False)

    # Define model, optimizer, and loss function
    num_steps = 1
    num_inputs = X_train_final.shape[2]
    output_size = y_train_final.shape[1]
    StudentModel = Net(num_inputs, num_steps).to(device)
    loss_fn = combined_loss

    optimizer = torch.optim.Adam(params=StudentModel.parameters(), lr=learning_rate)

    # put model into train mode
    StudentModel.train()

    # Train Loop
    for epoch in range(num_epochs):
        for i, (data, targets) in enumerate(iter(train_loader)):
            # move to device
            data = data.to(device)
            targets = targets.to(device)

            # change to floats
            data = data.float()
            targets = targets.float()

            # run forward pass
            _, _, _, mem = StudentModel(data)

            
            with torch.no_grad():
                y_teacher = targets[:, [3, 4, 5]].squeeze(-1)
                y_true = targets[:, [0, 1, 2]].squeeze(-1)
            
            y_pred = mem[-1]

            # Here, alter alpha from 0-1 to change between weighing quantile loss and distillation loss
            # Ideally, we would perform an alpha sweep to identify the optimized alpha value
            # calculate loss
            loss_val = loss_fn(y_true, y_pred, y_teacher, alpha=alpha)

            # calculate and store MAPE Loss
            mem_numpy = mem.cpu().detach().numpy()
            #mem_numpy = mem.detach().numpy()
            targets_numpy = y_true.cpu().detach().numpy()
            #targets_numpy = targets.detach().numpy()
            mape_hist.append(MAPE(mem_numpy[-1], targets_numpy))
            power_mape.append(MAPE(mem_numpy[-1][:,0], targets_numpy[:,0]))
            voltage_mape.append(MAPE(mem_numpy[-1][:,1], targets_numpy[:,1]))
            current_mape.append(MAPE(mem_numpy[-1][:,2], targets_numpy[:,2]))

            # Gradient calculation + weight update
            optimizer.zero_grad()
            loss_val.backward()
            optimizer.step()

            # Store loss history for future plotting
            loss_hist.append(loss_val.item())

            if len(loss_hist) > 100:
                avg_loss_hist.append(sum(loss_hist[-100:])/len(loss_hist[-100:]))
            else:
                avg_loss_hist.append(0)

        if len(loss_hist) > 100:
            print(f'Epoch {epoch}! Avg loss for the last 100 iterations: {avg_loss_hist[-1]}')

    # Define model, optimizer, and loss function
    num_steps = 1 
    num_inputs = X_train_final.shape[2]
    output_size = y_train_final.shape[1]

    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

    # Now let's test the model on the test set
    StudentModel.eval()  # Set model to evaluation mode
    actuals = []
    predictions = []

    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)

            # Get model predictions
            outputs = StudentModel(data)
            y_pred = outputs[-1]
            y_pred = y_pred.permute(1, 0, 2)  # Reorganize to match ground truth shape

            # Collect actual and predicted values
            actuals.append(targets.cpu().numpy()) 
            predictions.append(y_pred.cpu().numpy())

    # Concatenate all batches
    actuals = np.concatenate(actuals, axis=0)
    predictions = np.concatenate(predictions, axis=0)
    predictions = predictions.squeeze(1)

    # Compute MAPE for each output (e.g., power, voltage, current)
    mape_power = MAPE(actuals[:, 0], predictions[:, 0])
    mape_voltage = MAPE(actuals[:, 1], predictions[:, 1])
    mape_current = MAPE(actuals[:, 2], predictions[:, 2])

    # Print results
    print("Alpha = ", alpha)
    print(f"Test MAPE power Using Teacher-Student Model: {mape_power:.3f}")
    print(f"Test MAPE voltage Using Teacher-Student Model: {mape_voltage:.3f}")
    print(f"Test MAPE current Using Teacher-Student Model: {mape_current:.3f}")

Performing alpha sweep on student model
Epoch 1! Avg loss for the last 100 iterations: 3822594.6175
Epoch 2! Avg loss for the last 100 iterations: 3643008.33
Epoch 3! Avg loss for the last 100 iterations: 3472663.0225
Epoch 4! Avg loss for the last 100 iterations: 3309446.2425
Epoch 5! Avg loss for the last 100 iterations: 3152562.255
Epoch 6! Avg loss for the last 100 iterations: 3001566.7075
Epoch 7! Avg loss for the last 100 iterations: 2856154.6025
Epoch 8! Avg loss for the last 100 iterations: 2716089.975
Epoch 9! Avg loss for the last 100 iterations: 2581175.1575
Alpha =  0.0
Test MAPE power Using Teacher-Student Model: 0.745
Test MAPE voltage Using Teacher-Student Model: 0.601
Test MAPE current Using Teacher-Student Model: 0.805
Epoch 1! Avg loss for the last 100 iterations: 3434380.1325
Epoch 2! Avg loss for the last 100 iterations: 3267247.9025
Epoch 3! Avg loss for the last 100 iterations: 3108952.5475
Epoch 4! Avg loss for the last 100 iterations: 2957491.8025
Epoch 5! Avg l

In [ ]:
# Define model, optimizer, and loss function
num_steps = 1  # Since you're using LSTM for time series data with one step
num_inputs = X_train_final.shape[2]
output_size = y_train_final.shape[1]

optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Assuming the model has already been trained
# Now let's test the model on the test set

StudentModel.eval()
actuals = []
predictions = []


with torch.no_grad():
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)

        # Get model predictions
        outputs = StudentModel(data)
        y_pred = outputs[-1]
        y_pred = y_pred.permute(1, 0, 2)  # Reorganize to match ground truth shape

        # Collect actual and predicted values
        actuals.append(targets.cpu().numpy()) 
        predictions.append(y_pred.cpu().numpy())

# Concatenate all batches
actuals = np.concatenate(actuals, axis=0)
predictions = np.concatenate(predictions, axis=0)
predictions = predictions.squeeze(1)

# Compute MAPE for each output (e.g., power, voltage, current)
mape_power = MAPE(actuals[:, 0], predictions[:, 0])
mape_voltage = MAPE(actuals[:, 1], predictions[:, 1])
mape_current = MAPE(actuals[:, 2], predictions[:, 2])

# Print results
print(f"Test MAPE power Using Teacher-Student Model: {mape_power:.3f}")
print(f"Test MAPE voltage Using Teacher-Student Model: {mape_voltage:.3f}")
print(f"Test MAPE current Using Teacher-Student Model: {mape_current:.3f}")

Test MAPE power Using Teacher-Student Model: 0.491
Test MAPE voltage Using Teacher-Student Model: 0.607
Test MAPE current Using Teacher-Student Model: 0.808


## Concluding Thoughts
As we can see, the improvements made to the forecasting accuracy ie either currently non-existant or negligible. We we're hoping for improvements in the MAPE and percent error between the predicted and actual data, but in this iteration were not able to acheive meaningful improvements. 

One thing we noticed was that the components of the student model may have been suffering from overfitting, as lowering the number of epochs during the trianing process resulted in a significant improvement in the model accuracy. Training for 100 epochs led to mean absolute percent errors of 5+ in some prediciton categories, whereas training for 10 epochs minimized percent error in all categories.

### Kai added postnote 12/3/2024 22:45 PST
After some more slight alterations, and messing with alpha sweeps, Student-Teacher time relations, and hyperparamter tuning, I got much better results (not better than the original, but better than before) with regards to the MAPE between student predictions and actuals. I added an alpha sweep from 0 - 1 incrementing by 0.1, and found that each alpha value produced better results for each of power, voltage, and current, so it is difficult to determine which alpha value is most optimal. 

I can defintely say there is some merit to this approach, and this degradation in percent error could be due to the sporadicity of the given dataset. SMFC energy output is often not constant, and is difficult to predict as the power, voltage, and current can change drastically at any given moment. Studying the Stanford and UCSC datasets, it is not clear that the values follow any obvious pattern, and thus made this project slightly more difficult. Regardless, this has been a great learning experience and I am invested; this project will be seeing more of me this winter break.